# Notebook #4: Cálculo de Rentabilidad

- Con los datos extraídos y el modelo de *machine learning* que se ha entrenado para predecir el precio del alquiler, somos capaces de realizar el cálculo de rentabilidad con ayuda de distintas funciones.

- Importamos las librerías y soportes.

In [304]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los dfFrames

from tqdm import tqdm

# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Librería para el acceso a variables y funciones
# -----------------------------------------------------------------------
import sys
sys.path.append("../")
from src import soporte_rentabilidad as sr
from src import soporte_yolo as sy
from src import soporte_scoring as ss
from src import soporte_blip as sb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- Como primer paso, definimos una variable con las rutas de los transformers: el encoder, el scaler y el modelo Random Forest.

- Utilizaremos la función `predecir_alquiler` para aplicar el modelo a la totalidad de viviendas en venta, que devuelve el dataframe con una nueva columna 'alquiler_predicho'. Sus únicos argumentos son el archivo pickle con las viviendas en venta y la variable con las rutas de los transformers.

In [310]:
paths_transformers = ["../transformers/target_encoder.pkl", "../transformers/scaler.pkl", "../transformers/model.pkl"]
df = sr.predecir_alquiler("../data/transformed/final_sale.pkl", paths_transformers)

- Comprobamos el dataframe.

In [311]:
df.head(1)

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...",FINCAS RUIZ,876210884,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.86935 41.6973),Distrito Rural,1180.0


- Para cada vivienda, tenemos una columna que corresponde a una lista con las urls de las imágenes. Para poder identificar cuál corresponde a cada estancia, utilizaremos el modelo YOLOv11, en su versión x y de clasificación, por cuanto no nos interesa determinar la ubicación ni la forma de los objetos en las imágenes, solamente su existencia.

- Con la función `identificar_urls_habitaciones`, se añaden etiquetas a la cocina y al baño de cada anuncio. Para optimizar el tiempo de procesamiento, cuando las ha encontrado, no continua con el resto de imágenes, sino que salta al siguiente anuncio.

- El resultado es un dataframe con nuevas columnas para la URL del baño y de la cocina. Cuando uno de los dos valores es nulo, elimina esa fila. El return también incluye un dataframe con todas las detecciones de las imágenes procesadas, como una forma de poder validar los resultados.

In [312]:
df, detecciones = sy.identificar_urls_habitaciones(df, 'urls_imagenes')

100%|██████████| 365/365 [55:16<00:00,  9.09s/it] 


- Vamos a comprobar el resultado.

- Como una capa de verificación adicional, vamos a utilizar el modelo BLIP de Salesforce, que crea descripciones de los objetos, en este caso de las URLs identificadas por YOLO. Contamos luego el número de ocurrencias de 'kitchen' o 'bathroom'.

In [ ]:
captions_cocinas = sb.generar_descripciones(df, 'url_cocina')
sb.contar_palabras(captions_cocinas, 'kitchen')

- Una observación adicional a lo detectado en el EDA es que, aquellos anuncios en que hemos rellenado el valor nulo del anunciante con 'ND', corresponden generalmente a particulares, lo cual es relevante porque podría significar entorno a un 3% de ahorro en el valor de compra.

In [314]:
df[df['anunciante']=='ND'].sample(2)

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho,url_cocina,url_banio
87,106652992,89000.0,1780.0,piso,True,1,False,50.0,2,1,True,False,False,False,good,"calle Legión Gemina, 11",OPORTUNIDAD que no te puedes perder! PISO en v...,ND,608197088,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.92042 41.65292),Oliver-Valdefierro,835.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...
90,35110384,149000.0,1713.0,piso,True,1,False,87.0,3,1,False,True,True,False,good,"calle Goya, 10","Reforma integral: eléctricidad, tuberías sanit...",ND,625675251,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-1.0289 41.7224),Distrito Rural,891.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...


- El siguiente paso será utilizar la API de Anthropic, y específicamente el modelo Sonnet, para asignar una puntuación del 1 al 10 al estado de la estancia, y una estimación de los metros cuadrados, que es una consideración que ha surgido durante el desarrollo y la fase de pruebas del proyecto.

- El prompt que se utiliza incluye:

<div style="background-color:rgb(149, 222, 237); padding: 10px; border-left: 6px solid #000080; color: black; border-radius: 10px;">
You are an AI image analysis system specialized in evaluating property conditions. Your task is to analyze multiple property images in batch and provide a precise evaluation.
Instructions for Analysis:

Analyze each pair of images:

Image 1: Kitchen
Image 2: Bathroom

Evaluation Criteria:

Ratings: Whole numbers 1-5
1: Very poor (complete renovation required)
2: Poor (major renovations needed)
3: Fair (some renovations needed)
4: Good (minor improvements required)
5: Excellent (no renovations needed)

Sizes: Whole numbers in square meters (m²)
</div>

- Como puede verse en el prompt, el resultado de la consulta son 4 valores separados por comas, con la función `analizar_propiedades`, separaremos esos resultados y los asignaremos a 4 nuevas columnas del dataframe. Para tener consultas más eficientes, se enviarán en lotes, o *batches*, que corresponden a grupos de viviendas. Este batch más el dataframe con las URLs identificadas son los parámetros de la función.

In [138]:
df_sample = df.head(3)

In [186]:
df_sample, resultados = ss.analizar_propiedades(df_sample, batch=3)

Procesando lotes:   0%|          | 0/1 [00:00<?, ?it/s]

In [188]:
df_sample

,codigo,precio,precio_por_zona,tipo,exterior,planta,ascensor,tamanio,habitaciones,banios,aire_acondicionado,trastero,terraza,patio,estado,direccion,descripcion,anunciante,contacto,urls_imagenes,geometry,distrito,alquiler_predicho,url_cocina,url_banio,puntuacion_cocina,puntuacion_banio,mts_cocina,mts_banio
0,104792745,149900.0,1180.0,piso,True,4,True,127.0,3,2,False,False,False,True,good,"carretera de Huesca, 21","Junto a la Academia General Militar, en carret...",FINCAS RUIZ,876210884,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.86935 41.6973),Distrito Rural,1180.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,12,4
1,106086132,139900.0,1686.0,piso,True,3,True,83.0,2,1,False,False,True,False,good,calle de Portugal,Financiación hipoteca 100% ¡oportunidad única ...,ciz inmobiliaria,876210272,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.90232 41.65202),Delicias,761.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,4,12,4
2,105844791,149900.0,1162.0,piso,True,4,True,129.0,3,2,False,False,False,False,good,carretera de Huesca,Financiación hipoteca 100%. ¡oportunidad única...,ciz inmobiliaria,876210272,['https://img4.idealista.com/blur/WEB_LISTING-...,POINT (-0.87 41.69848),Distrito Rural,1186.0,https://img4.idealista.com/blur/WEB_LISTING-M/...,https://img4.idealista.com/blur/WEB_LISTING-M/...,4,5,12,4


In [317]:
df.to_pickle("../data/transformed/final_tags.pkl")

In [105]:
df_importado = pd.read_pickle("../data/transformed/final_tags.pkl")
df_importado.shape[0]

297